In [ ]:
import time
import threading
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client
import xmlrpc
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

server.register_introspection_functions()
stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")

# Добавление в лог через сервер
def add_log(log_line):
    try: stats_server.add_log(log_line)
    except Exception as e: 
        print("Не удалось сохранить лог: "+log_line+"\n"+str(e))
# Тест
def ping():
    add_log("ping")
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log("now")
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log("type")
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log("sum")
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log("pow")
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    add_log("black_list_by_name")
    frame = pd.read_csv("B:\\study\\labs1m\\DistributedProcessing\\LR1\\bad_boys2.csv", header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if exist:
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"
server.register_function(black_list_check, 'black_list_check')


# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check_fio_with_birthday(sname, name, patronym, birthday):
    add_log("black_list_by_fio")
    frame = pd.read_csv("B:\\study\\labs1m\\DistributedProcessing\\LR1\\bad_boys2.csv", header=0, sep=',', encoding='utf8')
    exist = len(frame[(frame['Surname']==sname) &
                      (frame['Name']==name) &
                      (frame['Patronym']==patronym) &
                      (frame['Birth']==birthday)]) != 0
    if exist:
        return sname +" "+ name +" "+ patronym + " ("+ birthday +"): "+ "bad_boy"
    else:
        return   sname +" "+ name +" "+ patronym + " ("+ birthday +"): "+ "good_boy"
server.register_function(black_list_check_fio_with_birthday, 'black_list_check_fio_with_birthday')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log("send_back_binary")
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Отправка изображения с сервера
def send_image(image_array):
    shape = image_array.shape
    if shape[2] == 1:
        image_array = image_array.reshape(shape[0], shape[1])
    pimg = pickle.dumps(image_array)        
    return xmlrpc.client.Binary(pimg)

# Чтение полученного изображения и настройка
def get_image_params(bin_img):
    img_arr = pickle.loads(bin_img.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    if img_arr.ndim == 3:
        channels = img_arr.shape[2]
    else:
        img_arr = img_arr.reshape(height, width, 1)
        channels = 1
    return [height, width, channels, img_arr]

# Инверсия цвета
# На вход изображение RGB размерности (M, N, C) со значениями 0-255
def send_back_inversion(bin_data):
    add_log("color_inversion")
    height, width, channels, img_arr = get_image_params(bin_data)
    for i in range(height):
        for j in range(width):
            for c in range(channels):
                img_arr[i][j][c] = 255 - img_arr[i][j][c]
    return send_image(img_arr)
server.register_function(send_back_inversion, 'color_inversion')


# Бинаризация изображения по порогу
def binarization_func(bin_data, value):
    add_log("image_binarization")
    height, width, channels, img_arr = get_image_params(bin_data)
    for i in range(height):
        for j in range(width):
            img_arr[i][j][:] = 0 if all(img_arr[i][j][:] < value) else 255
    return send_image(img_arr)
server.register_function(binarization_func, 'binarization_func')


# Поворот изображения по вертикали
def flip_vertical(bin_data):
    add_log("flip_vertical")
    height, width, channels, img_arr = get_image_params(bin_data)
    for i in range(height):
        for j in range(width//2):
            for c in range(channels):
                tmp = img_arr[i][j][c]
                img_arr[i][j][c] = img_arr[i][width-j-1][c]
                img_arr[i][width-j-1][c] = tmp
    return send_image(img_arr)
server.register_function(flip_vertical, 'flip_vertical')

print("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...
